# Evaluation Notebook for the AIZero Chess Bot

- Bot prediction from Fen string
- Bot plays against itself
- Bot vs Human
- Bot vs Baseline Bot

In [ ]:
%pip install torch chess numpy

In [ ]:
import time

from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Literal, Optional
from chess import Board, Move, WHITE, BLACK, Color


TIME_TO_THINK = 0.5  # seconds


class ChessBot(ABC):
    def __init__(self, name: str) -> None:
        """Initializes the bot with a name."""
        self.name = name
        self.start_time = 0.0

    @abstractmethod
    def think(self, board: Board) -> Move:
        """This method is called when it's the bot's turn to move. It should return the move that the bot wants to make."""
        raise NotImplementedError('Subclasses must implement this method')

    @property
    def time_elapsed(self) -> float:
        """Returns the time elapsed since the bot started thinking."""
        return time.time() - self.start_time

    @property
    def time_remaining(self) -> float:
        """
        Determines the time remaining for the bot to think.

        :return: The time remaining in seconds.
        """
        return TIME_TO_THINK - self.time_elapsed

    @property
    def time_is_up(self) -> bool:
        """Determines if the bot has run out of time to think."""
        return self.time_remaining <= 0

    def restart_clock(self) -> None:
        """Restarts the clock for the bot."""
        self.start_time = time.time()


@dataclass
class GameResult:
    winner: Optional[Color]
    result: Literal['1-0', '0-1', '1/2-1/2', 'unfinished']

    @staticmethod
    def from_board(board: Board) -> 'GameResult':
        if board.is_checkmate():
            result = '1-0' if board.turn == BLACK else '0-1'
            return GameResult(board.turn, result)

        if board.is_game_over():
            return GameResult(None, '1/2-1/2')

        return GameResult(None, 'unfinished')


class GameManager:
    def __init__(self, white: ChessBot, black: ChessBot) -> None:
        """Initializes the game manager with two players."""
        self.white = white
        self.black = black

    def play_game(self, verify_moves=True) -> GameResult:
        """Manages the gameplay loop until the game is over or a player quits."""
        board = Board()
        
        while not board.is_game_over():
            current_player = self.white if board.turn == WHITE else self.black

            current_player.restart_clock()
            move = current_player.think(board)

            if verify_moves and move not in board.legal_moves:
                raise ValueError(f'Invalid move {move} for player {current_player.name}')

            board.push(move)

        return GameResult.from_board(board)

In [ ]:
from chess import Board, Move
import chess.svg
from IPython.display import display, clear_output


class HumanPlayer(ChessBot):
    def __init__(self) -> None:
        """Initializes the human player."""
        super().__init__('Human')

    def think(self, board: Board) -> Move:
        """Allows a human player to input a move using the GUI."""
        self.show(board)
        
        print('Legal moves:', [move.uci() for move in board.legal_moves])
        while True:
            move = input('Enter your move: ')
            try:
                move = Move.from_uci(move)
                if move in board.legal_moves:
                    board.push(move)
                    self.show(board)
                    board.pop()
                    return move
                print('Invalid move. Try again.')
            except ValueError:
                print('Invalid move. Try again.')
        
    def show(self, board: Board) -> None:
        """Displays the current board state."""
        boardsvg = chess.svg.board(board, size=350)
        clear_output(wait=True)
        display(boardsvg)

In [ ]:
import chess.engine

class AlphaZeroBot(ChessBot):
    def __init__(self) -> None:
        super().__init__('Alpha MCTS Bot')
        time_to_think_ms = str(int(TIME_TO_THINK * 1000) + 5000) # Add 5 seconds to the time to think for startup time etc.
        self.engine = chess.engine.SimpleEngine.popen_uci(f'./EvalAIZeroChessBot "play" {time_to_think_ms}')

    def think(self, board: Board) -> Move:
        result = self.engine.play(board, chess.engine.Limit(time=TIME_TO_THINK))
        if result.move is None:
            raise ValueError('The engine returned a None move')
        return result.move
    
    def stop(self) -> None:
        self.engine.quit()

In [ ]:
import chess.engine

class BaselineBot(ChessBot):
    def __init__(self, engine_path: str, skill: int) -> None:
        super().__init__(f'Baseline Bot ({engine_path})')
        self.engine = chess.engine.SimpleEngine.popen_uci(engine_path)

        # Set the skill level of the engine
        # The skill level can be set from 0 to 20 (0 being the weakest and 20 the strongest)
        self.engine.configure({'Skill Level': skill}) 

    def think(self, board: Board) -> Move:
        result = self.engine.play(board, chess.engine.Limit(time=TIME_TO_THINK))
        if result.move is None:
            raise ValueError('The engine returned a None move')
        return result.move
    
    def stop(self) -> None:
        self.engine.quit()

In [ ]:
import subprocess

def runEvalChessBot(args: list[str]) -> list[str]:
    out = subprocess.run(['./EvalAIZeroChessBot'] + args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    # Wait for the process to finish
    out.check_returncode()    

    return [line.strip() for line in out.stdout.decode().split('\n') if line.strip()]

## Bot prediction from Fen string

In [ ]:
import chess.engine

def evaluate(fen: str) -> tuple[list[tuple[Move, float]], float]:
    lines = runEvalChessBot(['eval', fen])
    
    # The output is of the format:
    # "Position evaluation: " + value + "\n";
    # "Moves:" + "\n";
    # for (auto [move, probability] : moves) {
    #     move.uci() + " " + probability + "\n";
    # }
    
    while not lines[0].startswith('Position evaluation:'):
        lines.pop(0)
    
    value = float(lines[0].replace('Position evaluation: ', ''))
    moves = [(Move.from_uci(line.split(' ')[0]), float(line.split(' ')[1])) for line in lines[2:] if line]
    
    moves.sort(key=lambda x: x[1], reverse=True)
    
    return moves, value


stockfish = chess.engine.SimpleEngine.popen_uci('models/stockfish_8_x64')

mid_game_fens = [
    "r1bqkb1r/pp2pppp/2n2n2/2pp4/3PP3/2N2N2/PPP2PPP/R1BQKB1R w KQkq - 0 1",
    "r1bq1rk1/ppp1bppp/2np1n2/3Np3/1PP1P3/2N5/PB3PPP/R2QKB1R b KQ - 0 1",
    "r1bq1rk1/pppn1pbp/3p1np1/4p3/2PP4/2N2N2/PP2BPPP/R1BQ1RK1 w - - 0 1",
    "r2q1rk1/ppp1bppp/2np1n2/1B2p3/1bP1P3/2N2N2/PP1QBPPP/R4RK1 w - - 0 1",
    "2kr3r/ppp2ppp/2npb3/q7/3NP3/2N5/PPP1QPPP/R1B1K2R b KQ - 0 1",
    "r1bq1rk1/1pp1npbp/p2p1np1/3Pp3/2P1P3/2N1BN2/PP2BPPP/R2Q1RK1 w - - 0 1",
    "r2q1rk1/1bpp1ppp/p1n2n2/1p1pp3/3P1B2/2PBPN2/PP3PPP/R2QK2R w KQ - 0 1",
    "r2q1rk1/ppp2ppp/2nbpn2/3p4/3P1B2/2NBPN2/PPP2PPP/R2QK2R b KQ - 0 1",
    "rnbq1rk1/pp3pbp/3p1np1/2pPp3/2P1P3/2N2N2/PP3PPP/R1BQRBK1 b - - 0 1",
    "r1bq1rk1/pp2ppbp/2np1np1/8/2PNP3/2N5/PP3PPP/R1BQKB1R w KQ - 0 1"
]

for fen in mid_game_fens:
    moves_with_probabilities, value = evaluate(fen)
    board = Board(fen)
    stockfish_result = stockfish.analyse(board, chess.engine.Limit(time=0.1))

    print(f'FEN: {fen}')
    print(f'Value: {value:.4f}')
    print(f'Stockfish evaluation: {stockfish_result["score"]}')
    print(f'Moves with probabilities: {len(moves_with_probabilities)}')
    
    for move, probability in moves_with_probabilities:
        board.push(move)
        stockfish_result = stockfish.analyse(board, chess.engine.Limit(time=0.1))
        score = stockfish_result["score"].relative
        board.pop()
        
        print(f'{move}: NN: {probability:.4f} Stockfish: {score}')
        

## Bot plays against itself

In [ ]:
bot1 = AlphaZeroBot()
bot2 = AlphaZeroBot()

game_manager = GameManager(bot1, bot2)
game_manager.play_game()

## Bot vs Human

In [ ]:
bot = AlphaZeroBot()
human = HumanPlayer()

game_manager = GameManager(bot, human)
game_manager.play_game()

## Bot vs Baseline Bot

In [ ]:
bot = AlphaZeroBot()
baseline = BaselineBot('models/stockfish', skill=1)

game_manager = GameManager(bot, baseline)
result = game_manager.play_game()

baseline.stop()

print(result)

## Bot vs Baseline for all last checkpoints

In [ ]:
# import os
# 
# current_best_model_path = '...'
# best_model_iteration = int(current_best_model_path.split('_')[-1].replace('.pt', ''))
# 
# for i in range(1, best_model_iteration + 1):
#     model_path = f'models/model_{i}.pt'
#     if not os.path.exists(model_path):
#         continue
#     
#     bot_vs_baseline(model_path, repetitions=10, skill=1)

## Training Sample analyzer

Fetch one of the training sample batches and print out board positions and moves with predictions and targets.

In [ ]:
SAMPLE = "random" # or "312313812937128937912" some id in the MEMORY_DIR

lines = runEvalChessBot(['analyzeSample', SAMPLE])

print(lines)